In [2]:
#import requited libraries 
from requests import get
from bs4 import BeautifulSoup
from xlsxwriter import Workbook
import xlrd
import pandas as pd
import re
from timeit import default_timer as timer
import numpy as np

In [3]:
#check current working directory 
import os
print(os.getcwd())
os.chdir("C:\\Users\malejin\Documents\Python Scripts\__ingatlan_scraper")
print(os.getcwd())

C:\Users\malejin\Documents\Python Scripts\__ingatlan_scraper
C:\Users\malejin\Documents\Python Scripts\__ingatlan_scraper


In [4]:
url_base_list=["https://ingatlan.com",
               "https://dh.hu",
               "https://ingatlan.jofogas.hu/",
               "https://www.startlak.hu",
               "https://www.ingatlannet.hu/",
               "https://www.oc.hu/",
               "https://www.ingatlanbazar.hu/"
              ]
print(url_base_list)

['https://ingatlan.com', 'https://dh.hu', 'https://ingatlan.jofogas.hu/', 'https://www.startlak.hu', 'https://www.ingatlannet.hu/', 'https://www.oc.hu/', 'https://www.ingatlanbazar.hu/']


In [5]:
deal_type= input('kiadó vagy eladó: ')
building_type = input('lakas vagy haz: ')
location_county=input('megye: ')
location = input('hely: ')
min_price= input('minimum ár(ezer Ft): ')
max_price= input('maximum ár(ezer Ft): ')
min_sqm= input('minimum négyzetméter: ')
max_sqm= input('maximum_négyzetméter: ')
min_room_nr= input('minimum szobaszám: ')


kiadó vagy eladó: eladó
lakas vagy haz: ház
megye: Pest
hely: Dunakeszi
minimum ár(ezer Ft): 90
maximum ár(ezer Ft): 120
minimum négyzetméter: 90
maximum_négyzetméter: 200
minimum szobaszám: 3


In [6]:
print(min_room_nr)

3


In [164]:
'''url_base_list_city=["https://ingatlan.com/lista/elado+haz+dunakeszi",
                    "https://dh.hu/elado-ingatlan/haz/pest-megye/dunakeszi",
                    "https://ingatlan.jofogas.hu/pest/dunakeszi/haz?st=s",
                    "https://www.startlak.hu/dunakeszi+elado+haz-hazresz",
                    "https://www.ingatlannet.hu/elad%C3%B3/h%C3%A1z/Dunakeszi",
                    "https://www.oc.hu/ingatlanok/lista/felhasznalas:elado/tipus:house/hely-ertek:dunakeszi/hely-id:dunakeszi",
                    "https://www.ingatlanbazar.hu/ingatlan-Dunakeszi-Elado-Haz?property_location=1934&property__2=2"
                    
                   ]'''

'url_base_list_city=["https://ingatlan.com/lista/elado+haz+dunakeszi",\n                    "https://dh.hu/elado-ingatlan/haz/pest-megye/dunakeszi",\n                    "https://ingatlan.jofogas.hu/pest/dunakeszi/haz?st=s",\n                    "https://www.startlak.hu/dunakeszi+elado+haz-hazresz",\n                    "https://www.ingatlannet.hu/elad%C3%B3/h%C3%A1z/Dunakeszi",\n                    "https://www.oc.hu/ingatlanok/lista/felhasznalas:elado/tipus:house/hely-ertek:dunakeszi/hely-id:dunakeszi",\n                    "https://www.ingatlanbazar.hu/ingatlan-Dunakeszi-Elado-Haz?property_location=1934&property__2=2"\n                    \n                   ]'

In [165]:
ingatlancom_url="https://ingatlan.com/lista/"+str(deal_type)+"+"+str(building_type)+"+"+str(location).lower()+"+"+str(min_price)+"-"+str(max_price)+"-mFt"+"+"+str(min_sqm)+"-"+str(max_sqm)+"+"+str(min_room_nr)+"-szoba-felett"
ingatlancom_url

'https://ingatlan.com/lista/eladó+ház+dunakeszi+90-120-mFt+90-200+3-szoba-felett'

In [166]:
#https://dh.hu/elado-ingatlan/haz/pest-megye/dunakeszi/-/90-110-mFt/90-200-m2/3-szoba-tol
dh_url="https://dh.hu/"+str(deal_type).replace("ó","o")+"-ingatlan/"+str(building_type).replace("á","a")+"/"+str(location_county)+"-megye"+"/"+str(location)+"/-/"+str(min_price)+"-"+str(max_price)+"-mFt/"+str(min_sqm)+"-"+str(max_sqm)+"-m2/"+str(min_room_nr)+"-szoba-tol"
dh_url

'https://dh.hu/elado-ingatlan/haz/Pest-megye/Dunakeszi/-/90-120-mFt/90-200-m2/3-szoba-tol'

In [167]:
#https://ingatlan.jofogas.hu/pest/dunakeszi/haz?max_price=110000000&max_size=200&min_price=90000000&min_size=90&ros=3&st=s
jofogas_url="https://ingatlan.jofogas.hu/"+str(location_county).lower()+"/"+str(location).lower()+"?max_price="+str(max_price)+"000000"+"&max_size="+str(max_sqm)+"&min_price="+str(min_price)+"000000"+"&min_size="+str(min_sqm)+"&ros="+str(min_room_nr)+"&st=s"
jofogas_url

'https://ingatlan.jofogas.hu/pest/dunakeszi?max_price=120000000&max_size=200&min_price=90000000&min_size=90&ros=3&st=s'

In [168]:
#https://www.startlak.hu/dunakeszi+elado+haz-hazresz+ar-90000000-100000000+szoba-3-+alapterulet-90-200
startlak_url="https://www.startlak.hu/"+str(location).lower()+"+"+str(deal_type).replace("ó","o")+"+"+str(building_type).replace("á","a")+"-hazresz"+"+ar-"+str(min_price)+"-"+str(max_price)+"+szoba-"+str(min_room_nr)+"-+alapterulet-"+str(min_sqm)+"-"+str(max_sqm)
startlak_url

'https://www.startlak.hu/dunakeszi+elado+haz-hazresz+ar-90-120+szoba-3-+alapterulet-90-200'

In [169]:
#https://www.ingatlannet.hu/eladó/ház/Dunakeszi?minPrice=90&maxPrice=115&roomMin=3&plotSizeMin=90&plotSizeMax=200
ingatlannet_url="https://www.ingatlannet.hu/"+str(deal_type)+"/"+str(building_type)+"/"+str(location)+"?minPrice="+str(min_price)+"&maxPrice="+str(max_price)+"&roomMin="+str(min_room_nr)+"&plotSizeMin="+str(min_sqm)+"&plotSizeMax="+str(max_sqm)
ingatlannet_url

'https://www.ingatlannet.hu/eladó/ház/Dunakeszi?minPrice=90&maxPrice=120&roomMin=3&plotSizeMin=90&plotSizeMax=200'

In [170]:
#https://www.oc.hu/ingatlanok/lista/oldalszam:12/rendezes:relevance/felhasznalas:elado/tipus:house/hely-ertek:dunakeszi/hely-id:dunakeszi,/ar:90~110/netto-alapterulet:90~200/szobak:3~
oc_url="https://www.oc.hu/ingatlanok/lista/oldalszam:12/rendezes:relevance/felhasznalas:"+str(deal_type).replace("ó","o")+"/tipus:"+str(building_type).replace("ház", "house").replace("haz","house").replace("lakás","flat")+"/hely-ertek:"+str(location).lower()+"/hely-id:"+str(location).lower()+",/ar:"+str(min_price)+"~"+str(max_price)+"/netto-alapterulet:"+str(min_sqm)+"~"+str(max_sqm)+"/szobak:"+str(min_room_nr)+"~"
oc_url

'https://www.oc.hu/ingatlanok/lista/oldalszam:12/rendezes:relevance/felhasznalas:elado/tipus:house/hely-ertek:dunakeszi/hely-id:dunakeszi,/ar:90~120/netto-alapterulet:90~200/szobak:3~'

In [171]:
#https://www.ingatlanbazar.hu/ingatlan-Dunakeszi-Elado-Haz-90-110-millio-Ft-90m2-tol-200m2-ig-legfeljebb-3-szoba?property_location=1934&price_min=90&price_max=110&property_floor_area=90-200&property_rooms=3%2B&property__2=2
#ingatlanbazar_url="https://www.ingatlanbazar.hu/ingatlan-"+str(location).capitalize()+"-"+str(deal_type).capitalize()+"-"+str(building_type).capitalize()+"-"+str(min_price)+"-"+str(max_price)+"-millio-Ft-"+str(min_sqm)+"-tol-"+str(max_sqm)+"-ig-legfeljebb-"+str(min_room_nr)
#ingatlanbazar_url

In [172]:
list_urls=[ingatlancom_url,dh_url,jofogas_url,startlak_url, ingatlannet_url, oc_url]
list_urls

['https://ingatlan.com/lista/eladó+ház+dunakeszi+90-120-mFt+90-200+3-szoba-felett',
 'https://dh.hu/elado-ingatlan/haz/Pest-megye/Dunakeszi/-/90-120-mFt/90-200-m2/3-szoba-tol',
 'https://ingatlan.jofogas.hu/pest/dunakeszi?max_price=120000000&max_size=200&min_price=90000000&min_size=90&ros=3&st=s',
 'https://www.startlak.hu/dunakeszi+elado+haz-hazresz+ar-90-120+szoba-3-+alapterulet-90-200',
 'https://www.ingatlannet.hu/eladó/ház/Dunakeszi?minPrice=90&maxPrice=120&roomMin=3&plotSizeMin=90&plotSizeMax=200',
 'https://www.oc.hu/ingatlanok/lista/oldalszam:12/rendezes:relevance/felhasznalas:elado/tipus:house/hely-ertek:dunakeszi/hely-id:dunakeszi,/ar:90~120/netto-alapterulet:90~200/szobak:3~']

In [173]:
import requests # Import requests

# Windows 10 with Google Chrome
user_agent_desktop = '84.0.4147.105  (Windows NT 10.0; Win64; x64) '
headers = { 'User-Agent': user_agent_desktop}

soup_list=[]
for i in range(len(list_urls)):
    print(list_urls[i])
    resp = requests.get(list_urls[i], headers=headers)  # Send request
    #print(resp)
    code = resp.status_code  # HTTP response code
    #print(code)
    if code == 200:
        soup = BeautifulSoup(resp.text, 'lxml')  # Parsing the HTML
        #print(soup)
        soup_list.append(soup)
        #print(soup.prettify())
    else:
        print(f'Error to load accuweather.hu: {code}')

https://ingatlan.com/lista/eladó+ház+dunakeszi+90-120-mFt+90-200+3-szoba-felett
https://dh.hu/elado-ingatlan/haz/Pest-megye/Dunakeszi/-/90-120-mFt/90-200-m2/3-szoba-tol
https://ingatlan.jofogas.hu/pest/dunakeszi?max_price=120000000&max_size=200&min_price=90000000&min_size=90&ros=3&st=s
https://www.startlak.hu/dunakeszi+elado+haz-hazresz+ar-90-120+szoba-3-+alapterulet-90-200
https://www.ingatlannet.hu/eladó/ház/Dunakeszi?minPrice=90&maxPrice=120&roomMin=3&plotSizeMin=90&plotSizeMax=200
https://www.oc.hu/ingatlanok/lista/oldalszam:12/rendezes:relevance/felhasznalas:elado/tipus:house/hely-ertek:dunakeszi/hely-id:dunakeszi,/ar:90~120/netto-alapterulet:90~200/szobak:3~


In [174]:
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)

In [175]:
subpages_url_list=[]
price_list_per_sl=[]

In [176]:
#soup_list[0]

In [177]:
#get search list
ingatlancom_sl=soup_list[0].find('div', {'class': 'resultspage__listings js-listings'})
#print(ingatlancom_sl)

#get all hrefs from sl
links_with_text = []
for a in ingatlancom_sl.find_all('a', href=True): 
    if a.text: 
        if (a['href'][-8:].isdigit()):
            links_with_text.append("https://ingatlan.com"+a['href'])

links_with_text=list(set(links_with_text))            
subpages_url_list.append(links_with_text)
links_with_text

#i need only the ones with 8nums at the end

['https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/31768378',
 'https://ingatlan.com/dunakeszi/elado+haz/ikerhaz/32352879',
 'https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/31448814',
 'https://ingatlan.com/dunakeszi/elado+lakas/tegla-epitesu-lakas/32238592',
 'https://ingatlan.com/dunakeszi-alag/elado+haz/csaladi-haz/32188860',
 'https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/31953740',
 'https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/32335824',
 'https://ingatlan.com/dunakeszi-alag/elado+haz/csaladi-haz/32137729',
 'https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/32240337',
 'https://ingatlan.com/dunakeszi-alag/elado+haz/csaladi-haz/32392448',
 'https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/30444220',
 'https://ingatlan.com/dunakeszi-varoskozpont/elado+haz/csaladi-haz/32061495',
 'https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/32189689',
 'https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/31701880',
 'https://

In [178]:
len(links_with_text)

20

In [179]:
ingatlancom_price=ingatlancom_sl.find_all('div', {'class': 'price'})

ingatlancom_price_cleaned=[]
for i in range(len(ingatlancom_price)):
    ingatlancom_price_cleaned.append(striphtml(str(ingatlancom_price[i])).strip())

price_list_per_sl.append(ingatlancom_price_cleaned)   
ingatlancom_price_cleaned
#len(ingatlancom_price_cleaned)

['116 M Ft',
 '99 M Ft',
 '91.2 M Ft',
 '99 M Ft',
 '119 M Ft',
 '111 M Ft',
 '117 M Ft',
 '98.99 M Ft',
 '119 M Ft',
 '99 M Ft',
 '120 M Ft',
 '120 M Ft',
 '120 M Ft',
 '105 M Ft',
 '105 M Ft',
 '119.9 M Ft',
 '105 M Ft',
 '117 M Ft',
 '103 M Ft',
 '115 M Ft']

In [180]:
len(ingatlancom_price_cleaned)

20

In [181]:
soup_list[1]

<!DOCTYPE html>
<html>
<head>
<title>Eladó házak | Ingatlankereső | Duna House</title>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="IE=9" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, user-scalable=no" name="viewport"/>
<meta content="Eladó házak | Ingatlankereső | Duna House" property="og:title"/>
<meta content="Eladó házak | Ingatlankereső | Duna House" lang="hu" name="DC.Title"/>
<meta content="index,follow" name="robots"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="Eladó házak. Ingatlanok a dh.hu naprakész adatbázisából. Találja meg az otthont, amit keres!" name="description"/>
<meta content="Eladó házak. Ingatlanok a dh.hu naprakész adatbázisából. Találja meg az otthont, amit keres!" property="og:description"/>
<meta content="Eladó házak. Ingatlanok a dh.hu naprakész adatbázisából. Találja meg az ottho

In [182]:
'''#get search list
dh_sl=soup_list[1].find('div', {'class': 'propertiesVersions ver1'})
#print(type(dh_sl))
#print(dh_sl)

#get all hrefs from sl
links_with_text = []
for a in dh_sl.find_all('a', href=True): 
    if a.text: 
        if (a['href'][12:18].isdigit()):
            #print(a['href'][12:18])
            links_with_text.append("https://dh.hu"+a['href'])

#print(links_with_text)
links_with_text=list(set(links_with_text))
subpages_url_list.append(links_with_text)
links_with_text

#i need only the ones with 8nvarchars after ingatkan/'''

'#get search list\ndh_sl=soup_list[1].find(\'div\', {\'class\': \'propertiesVersions ver1\'})\n#print(type(dh_sl))\n#print(dh_sl)\n\n#get all hrefs from sl\nlinks_with_text = []\nfor a in dh_sl.find_all(\'a\', href=True): \n    if a.text: \n        if (a[\'href\'][12:18].isdigit()):\n            #print(a[\'href\'][12:18])\n            links_with_text.append("https://dh.hu"+a[\'href\'])\n\n#print(links_with_text)\nlinks_with_text=list(set(links_with_text))\nsubpages_url_list.append(links_with_text)\nlinks_with_text\n\n#i need only the ones with 8nvarchars after ingatkan/'

In [183]:
#len(links_with_text)

In [184]:
'''dh_price=dh_sl.find_all('div', {'class': 'price'})
#print(dh_price)
dh_price_cleaned=[]
for i in range(len(dh_price)):
    dh_price_cleaned.append(striphtml(str(dh_price[i])).strip())

price_list_per_sl.append(dh_price_cleaned)   
dh_price_cleaned'''

"dh_price=dh_sl.find_all('div', {'class': 'price'})\n#print(dh_price)\ndh_price_cleaned=[]\nfor i in range(len(dh_price)):\n    dh_price_cleaned.append(striphtml(str(dh_price[i])).strip())\n\nprice_list_per_sl.append(dh_price_cleaned)   \ndh_price_cleaned"

In [185]:
#len(dh_price_cleaned)

In [186]:
#soup_list[2]

In [187]:
#get search list
jofogas_sl=soup_list[2].find('div', {'class': 'main-box-body search-list-container'})
print(jofogas_sl)

#get all hrefs from sl
links_with_text = []
for a in jofogas_sl.find_all('a', href=True): 
    if a.text: 
        if (a['href'][-12:-4].isdigit()):
            #print(a['href'][-12:-4])
            links_with_text.append(a['href'])
            subpages_url_list.append

links_with_text=list(set(links_with_text))
subpages_url_list.append(links_with_text)
links_with_text

#i need only the ones wth 9nums before .htm

<div class="main-box-body search-list-container"> <div id="failover_msg_div">
</div> <div class="list-items"> <div class="col-xs-12 box listing list-item reListElement" data-tealium-ad-rank-sender="" itemprop="itemListElement" itemscope="" itemtype="http://schema.org/ListItem"> <meta content="1" itemprop="position"/>
<div class="realestate-item jfg-item reListItem uniq ribbon-item with-price-item discount-item company-item urgent-item" id="listid_118572234" itemprop="item" itemscope="" itemtype="http://schema.org/Product"> <div class="contentArea"> <section class="reLiSection imageBox"> <meta content="https://img.jofogas.hu/images/Dunakeszin_Egy_telken__Ket_haz__Tobb_generacios__769921960810390.jpg" itemprop="image"/> <a class="" data-adview-url="" href="https://ingatlan.jofogas.hu/pest/Dunakeszin_Egy_telken__Ket_haz__Tobb_generacios__118572234.htm" target="_self" title="Dunakeszin Egy telken, Két ház. Több generációs!, 1. Kép"> <picture class="li-default"> <source data-srcset="https:/

['https://ingatlan.jofogas.hu/pest/Elado_haz__Dunakeszi__99000000_Ft__161_nm_11560_bhv_103984875.htm',
 'https://ingatlan.jofogas.hu/pest/KETGENERACIOS_SORHAZ_ELADO__90514053.htm',
 'https://ingatlan.jofogas.hu/pest/Elado_Haz__Dunakeszi_120775939.htm',
 'https://ingatlan.jofogas.hu/pest/Elado_haz___Dunakeszi_115193316.htm',
 'https://ingatlan.jofogas.hu/pest/Elado_csaladi_haz_Dunakeszin_119261836.htm',
 'https://ingatlan.jofogas.hu/pest/Dunakeszin_EGY_telken_KET_haz__plusz_egy__kislak__118604331.htm',
 'https://ingatlan.jofogas.hu/pest/Dunakeszi_Alag_reszen_tagas_csaladi_haz__117986647.htm',
 'https://ingatlan.jofogas.hu/pest/Elado_Haz__Dunakeszi_117830347.htm',
 'https://ingatlan.jofogas.hu/pest/Dunakeszi_elado_tegla_ikerhaz_120668259.htm',
 'https://ingatlan.jofogas.hu/pest/Ritka_lehetoseg__ketgeneracios_sorhaz_Dunakeszin_elado__120046205.htm',
 'https://ingatlan.jofogas.hu/pest/Dunakeszi_elado_tegla_csaladi_haz_97044985.htm',
 'https://ingatlan.jofogas.hu/pest/Elado_haz__Dunakeszi__

In [188]:
jofogas_price=jofogas_sl.find_all('div', {'class': 'priceBox'})

jofogas_price_cleaned=[]
for i in range(len(jofogas_price)):
    jofogas_price_cleaned.append(striphtml(str(jofogas_price[i])).strip().replace("  \xa0"," "))

price_list_per_sl.append(jofogas_price_cleaned)   
jofogas_price_cleaned
#len(jofogas_price_cleaned)

['97 900 000 Ft',
 '106 900 000 Ft',
 '120 000 000 Ft',
 '119 000 000 Ft',
 '92 000 000 Ft',
 '95 000 000 Ft',
 '115 000 000 Ft',
 '99 000 000 Ft',
 '102 500 000 Ft',
 '99 000 000 Ft',
 '97 900 000 Ft',
 '120 000 000 Ft',
 '100 800 000 Ft',
 '106 900 000 Ft',
 '115 000 000 Ft',
 '99 000 000 Ft',
 '119 900 000 Ft',
 '115 000 000 Ft',
 '99 000 000 Ft',
 '95 000 000 Ft',
 '99 000 000 Ft',
 '100 000 000 Ft',
 '119 900 000 Ft']

In [189]:
soup_list[3]

<!DOCTYPE html>
<html lang="hu">
<head>
<!-- Google Tag Manager -->
<script data-cookieconsent="ignore">(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
		new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
		j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
		'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
		})(window,document,'script','dataLayer','GTM-5VB5XNC');</script>
<!-- End Google Tag Manager -->
<base href="https://www.startlak.hu"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" name="viewport"/>
<title>Fényképes eladó házak Dunakeszin részletes leírással |Startlak</title>
<meta content="Eladó házat keresel? A Startlakon 30 db fényképes dunakeszi eladó ház - házrész hirdetés vár. Az eladó házak között többféle kategória megtalálható és persze Dunakeszin városrész szerint is szűrhetjük a találat

In [190]:
#get search list
stratlak_sl=soup_list[3].find_all('div', {'class': 'cardData row col-md-8 col-lg-8'})
#print(stratlak_sl)

#get all hrefs from sl
links_with_text = []
for i in range(len(stratlak_sl)):
    for a in stratlak_sl[i].find_all('a', href=True): 
        if a.text: 
            #print(a.text)
            links_with_text.append("https://www.startlak.hu/"+a['href'])

links_with_text=list(set(links_with_text))
subpages_url_list.append(links_with_text)
links_with_text

['https://www.startlak.hu//ingatlan/elado+csaladi-haz+dunakeszi/7101450',
 'https://www.startlak.hu//ingatlan/elado+csaladi-haz+dunakeszi+dozsa-gyorgy-ter/7050809',
 'https://www.startlak.hu//ingatlan/elado+ikerhaz+dunakeszi/7027163',
 'https://www.startlak.hu//ingatlan/elado+csaladi-haz+dunakeszi+rev-utca/7066506',
 'https://www.startlak.hu//ingatlan/elado+csaladi-haz+dunakeszi/6970803',
 'https://www.startlak.hu//ingatlan/elado+csaladi-haz+dunakeszi+kert-utca/7074438',
 'https://www.startlak.hu//ingatlan/elado+csaladi-haz+dunakeszi/7004497',
 'https://www.startlak.hu//ingatlan/elado+ikerhaz+dunakeszi/7087749',
 'https://www.startlak.hu//ingatlan/elado+csaladi-haz+dunakeszi/6880560',
 'https://www.startlak.hu//ingatlan/elado+csaladi-haz+dunakeszi/6945341',
 'https://www.startlak.hu//ingatlan/elado+csaladi-haz+dunakeszi/7004494',
 'https://www.startlak.hu//ingatlan/elado+csaladi-haz+dunakeszi+kert-utca/7066505',
 'https://www.startlak.hu//ingatlan/elado+csaladi-haz+dunakeszi+dunakeszi/

In [191]:
len(links_with_text)

20

In [192]:
stratlak_price=[]
for i in range(len(stratlak_sl)):
    #print(stratlak_sl[i])
    #print(stratlak_sl[i].find_all('div', {'class': 'cardPrice hidden-xs hidden-sm'}))
    stratlak_price.append(striphtml(str(stratlak_sl[i].find_all('div', {'class': 'cardPrice hidden-xs hidden-sm'}))).replace("'","").replace("[","").replace("]",""))
    #price_list_per_sl.append(striphtml(str(stratlak_sl[i].find_all('div', {'class': 'cardPrice hidden-xs hidden-sm'}))).replace("'","").replace("[","").replace("]",""))
stratlak_price
price_list_per_sl.append(stratlak_price)
stratlak_price




['98 000 000 Ft',
 '98 000 000 Ft',
 '119 000 000 Ft',
 '113 900 000 Ft',
 '106 900 000 Ft',
 '106 900 000 Ft',
 '99 000 000 Ft',
 '98 990 000 Ft',
 '113 900 000 Ft',
 '106 900 000 Ft',
 '102 500 000 Ft',
 '99 000 000 Ft',
 '97 900 000 Ft',
 '99 000 000 Ft',
 '120 000 000 Ft',
 '106 900 000 Ft',
 '115 000 000 Ft',
 '117 300 000 Ft',
 '99 000 000 Ft',
 '99 000 000 Ft']

In [193]:
len(stratlak_price)

20

In [194]:
#soup_list[4]

In [195]:
#get search list
ingatlannet_sl=soup_list[4].find('div', {'class': 'list-view'})
#print(ingatlannet_sl)

#get all hrefs from sl
links_with_text = []
for a in ingatlannet_sl.find_all('a', href=True): 
    if a.text: 
        if (a['href'][20:27].isdigit()):
            #print(a['href'][20:27])
            links_with_text.append("https://www.ingatlannet.hu"+a['href'])
        
links_with_text=list(set(links_with_text))
subpages_url_list.append(links_with_text)
links_with_text

['https://www.ingatlannet.hu/ingatlan/Dunakeszi/5896320-eladó-családi-ház',
 'https://www.ingatlannet.hu/ingatlan/Dunakeszi/5932327-eladó-családi-ház',
 'https://www.ingatlannet.hu/ingatlan/Dunakeszi/6127294-eladó-Dunakeszi-családi-ház',
 'https://www.ingatlannet.hu/ingatlan/Dunakeszi/5964859-eladó-Tavasz-utca-családi-ház',
 'https://www.ingatlannet.hu/ingatlan/Dunakeszi/5635253-eladó-ikerház',
 'https://www.ingatlannet.hu/ingatlan/Dunakeszi/5879489-eladó-Jácint-utca-családi-ház',
 'https://www.ingatlannet.hu/ingatlan/Dunakeszi/6048787-eladó-családi-ház',
 'https://www.ingatlannet.hu/ingatlan/Dunakeszi/6091822-eladó-Dunakeszi-Duna-part-közelében-családi-ház',
 'https://www.ingatlannet.hu/ingatlan/Dunakeszi/6114947-eladó-ikerház',
 'https://www.ingatlannet.hu/ingatlan/Dunakeszi/5607781-eladó-családi-ház',
 'https://www.ingatlannet.hu/ingatlan/Dunakeszi/5728517-eladó-családi-ház']

In [196]:
ingatlannet_price=ingatlannet_sl.find_all('p', {'class': 'h2 d-none d-md-block price-text mt-3 mb-3'})

ingatlannet_price_cleaned=[]
for i in range(len(ingatlannet_price)):
    ingatlannet_price_cleaned.append(striphtml(str(ingatlannet_price[i])).strip().replace("  \xa0"," "))

price_list_per_sl.append(ingatlannet_price_cleaned)   
type(ingatlannet_price_cleaned[0])
#len(ingatlannet_price_cleaned)

str

In [197]:
#soup_list[5]

In [198]:
#get search list
oc_sl=soup_list[5].find('div', {'class': 'estate-list'})
#print(oc_sl)

#get all hrefs from sl
links_with_text = []
for a in oc_sl.find_all('a', href=True): 
    if a.text: 
        links_with_text.append("https://www.oc.hu"+a['href'])

links_with_text=list(set(links_with_text))
subpages_url_list.append(links_with_text)
links_with_text

['https://www.oc.hu/ingatlanok/H434067?c=963a6bdf',
 'https://www.oc.hu/ingatlanok/H430331?c=963a6bdf',
 'https://www.oc.hu/ingatlanok/H381282?c=963a6bdf',
 'https://www.oc.hu/ingatlanok/DHZ068165?c=963a6bdf',
 'https://www.oc.hu/ingatlanok/H404836?c=963a6bdf',
 'https://www.oc.hu/ingatlanok/DHZ520994?c=963a6bdf',
 'https://www.oc.hu/ingatlanok/H430043?c=963a6bdf',
 'https://www.oc.hu/ingatlanok/DHZ049373?c=963a6bdf']

In [199]:
oc_price=oc_sl.find_all('span', {'class': 'estate-price atEstateListItem-Price col-xs-6'})

oc_price_cleaned=[]
for i in range(len(oc_price)):
    oc_price_cleaned.append(striphtml(str(oc_price[i])).strip().replace("  \xa0"," "))

price_list_per_sl.append(oc_price_cleaned)   
oc_price_cleaned
#len(oc_price_cleaned)

['119 M Ft',
 '102,5 M Ft',
 '115 M Ft',
 '106,9 M Ft',
 '113,9 M Ft',
 '99 M Ft',
 '95 M Ft',
 '97,9 M Ft']

In [200]:
subpages_url_list

[['https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/31768378',
  'https://ingatlan.com/dunakeszi/elado+haz/ikerhaz/32352879',
  'https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/31448814',
  'https://ingatlan.com/dunakeszi/elado+lakas/tegla-epitesu-lakas/32238592',
  'https://ingatlan.com/dunakeszi-alag/elado+haz/csaladi-haz/32188860',
  'https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/31953740',
  'https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/32335824',
  'https://ingatlan.com/dunakeszi-alag/elado+haz/csaladi-haz/32137729',
  'https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/32240337',
  'https://ingatlan.com/dunakeszi-alag/elado+haz/csaladi-haz/32392448',
  'https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/30444220',
  'https://ingatlan.com/dunakeszi-varoskozpont/elado+haz/csaladi-haz/32061495',
  'https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/32189689',
  'https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/3170188

In [201]:
subpages_url_list_flat=[]
for i in range(len(subpages_url_list)):
    #print(subpages_url_list[i])
    for j in range(len(subpages_url_list[i])):
        print(subpages_url_list[i][j])
        subpages_url_list_flat.append(subpages_url_list[i][j])
len(subpages_url_list_flat)      

https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/31768378
https://ingatlan.com/dunakeszi/elado+haz/ikerhaz/32352879
https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/31448814
https://ingatlan.com/dunakeszi/elado+lakas/tegla-epitesu-lakas/32238592
https://ingatlan.com/dunakeszi-alag/elado+haz/csaladi-haz/32188860
https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/31953740
https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/32335824
https://ingatlan.com/dunakeszi-alag/elado+haz/csaladi-haz/32137729
https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/32240337
https://ingatlan.com/dunakeszi-alag/elado+haz/csaladi-haz/32392448
https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/30444220
https://ingatlan.com/dunakeszi-varoskozpont/elado+haz/csaladi-haz/32061495
https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/32189689
https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/31701880
https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/32359

82

In [202]:
price_list_per_sl
price_list_per_sl_flat=[]
for i in range(len(price_list_per_sl)):
    #print(price_list_per_sl[i])
    for j in range(len(price_list_per_sl[i])):
        #print(price_list_per_sl[i][j])
        price_list_per_sl_flat.append(price_list_per_sl[i][j])
len(price_list_per_sl_flat)

82

In [203]:
import requests # Import requests

# Windows 10 with Google Chrome
user_agent_desktop = '84.0.4147.105  (Windows NT 10.0; Win64; x64) '
headers = { 'User-Agent': user_agent_desktop}

detailed_page_url=[]
soup_list=[]
for i in range(len(subpages_url_list)):
    #print(subpages_url_list[i])
    for j in range(len(subpages_url_list[i])):
        #print(subpages_url_list[i][j])
        detailed_page_url.append(subpages_url_list[i][j])
        resp = requests.get(subpages_url_list[i][j], headers=headers)  # Send request
        #print(resp)
        code = resp.status_code  # HTTP response code
        #print(code)
        if code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')  # Parsing the HTML
            #print(soup)
            soup_list.append(soup)
            #print(soup.prettify())
        else:
            print(subpages_url_list[i][j])
            print(f'Error to load page: {code}')
    

In [204]:
len(soup_list)

82

In [205]:
soup_list[0]

<!DOCTYPE html>
<html data-n-head="%7B%22lang%22:%7B%22ssr%22:%22hu%22%7D%7D" data-n-head-ssr="" lang="hu">
<head>
<title>Eladó családi ház - Dunakeszi, Pest megye #31768378</title><meta content="width=device-width, initial-scale=1" data-n-head="ssr" name="viewport"/><meta charset="utf-8" data-hid="charset" data-n-head="ssr"/><meta content="yes" data-hid="mobile-web-app-capable" data-n-head="ssr" name="mobile-web-app-capable"/><meta content="ingatlan.com - Mindenhol jó, de a legjobb itt vár rád." data-hid="apple-mobile-web-app-title" data-n-head="ssr" name="apple-mobile-web-app-title"/><meta content="#d35630" data-hid="theme-color" data-n-head="ssr" name="theme-color"/><meta content="website" data-hid="og:type" data-n-head="ssr" name="og:type" property="og:type"/><meta content="ingatlan.com - Mindenhol jó, de a legjobb itt vár rád." data-hid="og:site_name" data-n-head="ssr" name="og:site_name" property="og:site_name"/><meta content="512" data-hid="og:image:width" data-n-head="ssr" name

In [295]:
ingatlan_df=pd.DataFrame(columns=['source_page','detailed_page_url','price','building_year','is_train', 'is_bus', 'is_any_pt','garage','is_hs_gas', 'garden_size_m2', 'is_nursery_near', 'is_school_near','street'])
ingatlan_df['detailed_page_url']=detailed_page_url
ingatlan_df

,source_page,detailed_page_url,price,building_year,is_train,is_bus,is_any_pt,garage,is_hs_gas,garden_size_m2,is_nursery_near,is_school_near,street
0,NaN,https://ingatlan.com/dunakeszi/elado+haz/csala...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,https://ingatlan.com/dunakeszi/elado+haz/ikerh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,https://ingatlan.com/dunakeszi/elado+haz/csala...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,https://ingatlan.com/dunakeszi/elado+lakas/teg...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,https://ingatlan.com/dunakeszi-alag/elado+haz/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,NaN,https://www.oc.hu/ingatlanok/DHZ068165?c=963a6bdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,NaN,https://www.oc.hu/ingatlanok/H404836?c=963a6bdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,NaN,https://www.oc.hu/ingatlanok/DHZ520994?c=963a6bdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,NaN,https://www.oc.hu/ingatlanok/H430043?c=963a6bdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [207]:
len(ingatlan_df)

82

In [208]:
#ADDING SOURCE PAGE
page=[]
for i in range(len(ingatlan_df)):
    #print(str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0])
    page.append(str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0])
ingatlan_df['source_page']=page
ingatlan_df

,source_page,detailed_page_url,price,building_year,is_train,is_bus,is_any_pt,garage,is_hs_gas,garden_size,is_nursery_near,is_school_near,street
0,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/ikerh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ingatlan.com,https://ingatlan.com/dunakeszi/elado+lakas/teg...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ingatlan.com,https://ingatlan.com/dunakeszi-alag/elado+haz/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ068165?c=963a6bdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,www.oc.hu,https://www.oc.hu/ingatlanok/H404836?c=963a6bdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ520994?c=963a6bdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,www.oc.hu,https://www.oc.hu/ingatlanok/H430043?c=963a6bdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [209]:
#ADDING PRICE
ingatlan_df['price']=price_list_per_sl_flat
ingatlan_df

,source_page,detailed_page_url,price,building_year,is_train,is_bus,is_any_pt,garage,is_hs_gas,garden_size,is_nursery_near,is_school_near,street
0,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,116 M Ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/ikerh...,99 M Ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,91.2 M Ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ingatlan.com,https://ingatlan.com/dunakeszi/elado+lakas/teg...,99 M Ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ingatlan.com,https://ingatlan.com/dunakeszi-alag/elado+haz/...,119 M Ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ068165?c=963a6bdf,"106,9 M Ft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,www.oc.hu,https://www.oc.hu/ingatlanok/H404836?c=963a6bdf,"113,9 M Ft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ520994?c=963a6bdf,99 M Ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,www.oc.hu,https://www.oc.hu/ingatlanok/H430043?c=963a6bdf,95 M Ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [210]:
#CHECKING IF ANY OF THE PUBLIC TRANSPORT FORMS ARE MENTIONED IN THE DESCRIPTION
is_train=[]
for i in range(len(soup_list)):
    #print(soup_list[i])
    mystring = striphtml(str(soup_list[i]).lower())
    #print(mystring)
    public_transport=re.compile(r'(vasút.*)(?![^<>]*>)') 
    public_transport=public_transport.findall(mystring)
    if public_transport!=[]:
        is_train.append("yes")
    else:
        is_train.append(np.nan)
    
len(is_train)

82

In [211]:
ingatlan_df['is_train']=is_train
ingatlan_df
#ingatlan_df[ingatlan_df['is_train']=='yes']

,source_page,detailed_page_url,price,building_year,is_train,is_bus,is_any_pt,garage,is_hs_gas,garden_size,is_nursery_near,is_school_near,street
0,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,116 M Ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/ikerh...,99 M Ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,91.2 M Ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ingatlan.com,https://ingatlan.com/dunakeszi/elado+lakas/teg...,99 M Ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ingatlan.com,https://ingatlan.com/dunakeszi-alag/elado+haz/...,119 M Ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ068165?c=963a6bdf,"106,9 M Ft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,www.oc.hu,https://www.oc.hu/ingatlanok/H404836?c=963a6bdf,"113,9 M Ft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ520994?c=963a6bdf,99 M Ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,www.oc.hu,https://www.oc.hu/ingatlanok/H430043?c=963a6bdf,95 M Ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [212]:
is_bus=[]
for i in range(len(soup_list)):
    #print(soup_list[i])
    mystring = striphtml(str(soup_list[i]).lower())
    #print(mystring)
    public_transport=re.compile(r'(busz.*)(?![^<>]*>)') 
    public_transport=public_transport.findall(mystring)
    if public_transport!=[]:
        is_bus.append("yes")
    else:
        is_bus.append(np.nan)
    
len(is_bus)

82

In [213]:
ingatlan_df['is_bus']=is_bus
ingatlan_df
#ingatlan_df[ingatlan_df['is_bus']=='yes']

,source_page,detailed_page_url,price,building_year,is_train,is_bus,is_any_pt,garage,is_hs_gas,garden_size,is_nursery_near,is_school_near,street
0,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,116 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/ikerh...,99 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,91.2 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ingatlan.com,https://ingatlan.com/dunakeszi/elado+lakas/teg...,99 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ingatlan.com,https://ingatlan.com/dunakeszi-alag/elado+haz/...,119 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ068165?c=963a6bdf,"106,9 M Ft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,www.oc.hu,https://www.oc.hu/ingatlanok/H404836?c=963a6bdf,"113,9 M Ft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ520994?c=963a6bdf,99 M Ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,www.oc.hu,https://www.oc.hu/ingatlanok/H430043?c=963a6bdf,95 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [214]:
is_any_pt=[]
for i in range(len(soup_list)):
    #print(soup_list[i])
    mystring = striphtml(str(soup_list[i]).lower())
    #print(mystring)
    public_transport=re.compile(r'(tömegközlekedés.*)(?![^<>]*>)') 
    public_transport=public_transport.findall(mystring)
    if public_transport!=[]:
        is_any_pt.append("yes")
    else:
        is_any_pt.append(np.nan)
    
len(is_any_pt)

82

In [215]:
ingatlan_df['is_any_pt']=is_any_pt
ingatlan_df
#ingatlan_df[ingatlan_df['is_any_pt']=='yes']

,source_page,detailed_page_url,price,building_year,is_train,is_bus,is_any_pt,garage,is_hs_gas,garden_size,is_nursery_near,is_school_near,street
0,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,116 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/ikerh...,99 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,91.2 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ingatlan.com,https://ingatlan.com/dunakeszi/elado+lakas/teg...,99 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ingatlan.com,https://ingatlan.com/dunakeszi-alag/elado+haz/...,119 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ068165?c=963a6bdf,"106,9 M Ft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,www.oc.hu,https://www.oc.hu/ingatlanok/H404836?c=963a6bdf,"113,9 M Ft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ520994?c=963a6bdf,99 M Ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,www.oc.hu,https://www.oc.hu/ingatlanok/H430043?c=963a6bdf,95 M Ft,NaN,NaN,yes,yes,NaN,NaN,NaN,NaN,NaN,NaN


In [216]:
is_nursery_near=[]
for i in range(len(soup_list)):
    #print(soup_list[i])
    mystring = striphtml(str(soup_list[i]).lower())
    #print(mystring)
    nursery_near=re.compile(r'(óvoda.*)(?![^<>]*>)') 
    nursery_near=nursery_near.findall(mystring)
    if nursery_near!=[]:
        is_nursery_near.append("yes")
    else:
        is_nursery_near.append(np.nan)
    
len(is_nursery_near)

82

In [217]:
ingatlan_df['is_nursery_near']=is_nursery_near
ingatlan_df


,source_page,detailed_page_url,price,building_year,is_train,is_bus,is_any_pt,garage,is_hs_gas,garden_size,is_nursery_near,is_school_near,street
0,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,116 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,yes,NaN,NaN
1,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/ikerh...,99 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,yes,NaN,NaN
2,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,91.2 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,yes,NaN,NaN
3,ingatlan.com,https://ingatlan.com/dunakeszi/elado+lakas/teg...,99 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,yes,NaN,NaN
4,ingatlan.com,https://ingatlan.com/dunakeszi-alag/elado+haz/...,119 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,yes,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ068165?c=963a6bdf,"106,9 M Ft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,www.oc.hu,https://www.oc.hu/ingatlanok/H404836?c=963a6bdf,"113,9 M Ft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ520994?c=963a6bdf,99 M Ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,www.oc.hu,https://www.oc.hu/ingatlanok/H430043?c=963a6bdf,95 M Ft,NaN,NaN,yes,yes,NaN,NaN,NaN,yes,NaN,NaN


In [218]:
is_school_near=[]
for i in range(len(soup_list)):
    #print(soup_list[i])
    mystring = striphtml(str(soup_list[i]).lower())
    #print(mystring)
    school_near=re.compile(r'(iskola.*)(?![^<>]*>)') 
    school_near=school_near.findall(mystring)
    if school_near!=[]:
        is_school_near.append("yes")
    else:
        is_school_near.append(np.nan)
    
len(is_school_near)

82

In [219]:
ingatlan_df['is_school_near']=is_school_near
ingatlan_df
#ingatlan_df[ingatlan_df['is_school_near']=='yes']

,source_page,detailed_page_url,price,building_year,is_train,is_bus,is_any_pt,garage,is_hs_gas,garden_size,is_nursery_near,is_school_near,street
0,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,116 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,yes,yes,NaN
1,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/ikerh...,99 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,yes,yes,NaN
2,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,91.2 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,yes,yes,NaN
3,ingatlan.com,https://ingatlan.com/dunakeszi/elado+lakas/teg...,99 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,yes,yes,NaN
4,ingatlan.com,https://ingatlan.com/dunakeszi-alag/elado+haz/...,119 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,yes,yes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ068165?c=963a6bdf,"106,9 M Ft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,www.oc.hu,https://www.oc.hu/ingatlanok/H404836?c=963a6bdf,"113,9 M Ft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ520994?c=963a6bdf,99 M Ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,www.oc.hu,https://www.oc.hu/ingatlanok/H430043?c=963a6bdf,95 M Ft,NaN,NaN,yes,yes,NaN,NaN,NaN,yes,yes,NaN


In [220]:
street=[]
for i in range(len(soup_list)):
    #print(soup_list[i])
    mystring = striphtml(str(soup_list[i]).lower())
    #print(mystring)
    street_op=re.compile(r'(utca.*)(?![^<>]*>)') 
    street_op=street_op.findall(mystring)
    get_street=mystring.split("utca")[0]
    if street_op!=[]:
        #print((get_street[-30:]+"utca"))
        street.append((get_street[-50:]+"utca"))
    else:
        street.append(np.nan)
    
street



['őségei: ingatlan.com zrt; 1016 budapest, mészáros utca',
 '\n\n\neladó ikerház - dunakeszi, lázár vilmos utca',
 '\n\n\neladó családi ház - dunakeszi, pozsonyi utca',
 'tér\n   \n    nincs megadva\n  \n    parkolás\n   \n    utca',
 'őségei: ingatlan.com zrt; 1016 budapest, mészáros utca',
 'szi eszter","officename":"otthon centrum erzsébet utca',
 'ció zavartalan együttélését is lehetővé teszi. az utca',
 'őségei: ingatlan.com zrt; 1016 budapest, mészáros utca',
 'ció zavartalan együttélését is lehetővé teszi. az utca',
 'őségei: ingatlan.com zrt; 1016 budapest, mészáros utca',
 '\n\n\neladó családi ház - dunakeszi, juhász gyula utca',
 'őségei: ingatlan.com zrt; 1016 budapest, mészáros utca',
 'őségei: ingatlan.com zrt; 1016 budapest, mészáros utca',
 'őségei: ingatlan.com zrt; 1016 budapest, mészáros utca',
 'őségei: ingatlan.com zrt; 1016 budapest, mészáros utca',
 'őségei: ingatlan.com zrt; 1016 budapest, mészáros utca',
 'őségei: ingatlan.com zrt; 1016 budapest, mészáros utca',

In [221]:
street_cleaned=[]
for j in range(len(street)):  
    if str(location).lower() in str(street[j]).lower():
        print(str(street[j].rsplit(",")[-1:]).replace("['","").replace("']",""))
        
        street_cleaned.append(str(street[j].rsplit(",")[-1:]).replace("['","").replace("']",""))
    else:
        print(np.nan)
        street_cleaned.append(np.nan)
len(street_cleaned)

nan
 lázár vilmos utca
 pozsonyi utca
nan
nan
nan
nan
nan
nan
nan
 juhász gyula utca
nan
nan
nan
nan
nan
nan
nan
nan
 jászai mari utca
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
 juhász gyula utca
nan
nan
nan
 rév utca
nan
 kert utca
nan
nan
nan
nan
nan
 kert utca
nan
nan
nan
nan
nan
 rév utca
nan
nan
 jácint utca
 jácint utca
 jácint utca
 tavasz utca
nan
 jácint utca
 jácint utca
 jácint utca
nan
 jácint utca
 jácint utca
nan
nan
 bazsanth vince utca
nan
nan
 bazsanth vince utca
nan
 bazsanth vince utca


82

In [223]:
ingatlan_df['street']=street_cleaned
ingatlan_df

,source_page,detailed_page_url,price,building_year,is_train,is_bus,is_any_pt,garage,is_hs_gas,garden_size,is_nursery_near,is_school_near,street
0,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,116 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,yes,yes,NaN
1,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/ikerh...,99 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,yes,yes,lázár vilmos utca
2,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,91.2 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,yes,yes,pozsonyi utca
3,ingatlan.com,https://ingatlan.com/dunakeszi/elado+lakas/teg...,99 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,yes,yes,NaN
4,ingatlan.com,https://ingatlan.com/dunakeszi-alag/elado+haz/...,119 M Ft,NaN,NaN,yes,NaN,NaN,NaN,NaN,yes,yes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ068165?c=963a6bdf,"106,9 M Ft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,www.oc.hu,https://www.oc.hu/ingatlanok/H404836?c=963a6bdf,"113,9 M Ft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ520994?c=963a6bdf,99 M Ft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bazsanth vince utca
80,www.oc.hu,https://www.oc.hu/ingatlanok/H430043?c=963a6bdf,95 M Ft,NaN,NaN,yes,yes,NaN,NaN,NaN,yes,yes,NaN


In [453]:
'''https://ingatlan.com/lista/eladó+ház+dunakeszi+90-120-mFt+90-200+3-szoba-felett
https://dh.hu/elado-ingatlan/haz/Pest-megye/Dunakeszi/-/90-120-mFt/90-200-m2/3-szoba-tol
https://ingatlan.jofogas.hu/pest/dunakeszi?max_price=120000000&max_size=200&min_price=90000000&min_size=90&ros=3&st=s
https://www.startlak.hu/dunakeszi+elado+haz-hazresz+ar-90-120+szoba-3-+alapterulet-90-200
https://www.ingatlannet.hu/eladó/ház/Dunakeszi?minPrice=90&maxPrice=120&roomMin=3&plotSizeMin=90&plotSizeMax=200
https://www.oc.hu/ingatlanok/lista/oldalszam:12/rendezes:relevance/felhasznalas:elado/tipus:house/hely-ertek:dunakeszi/hely-id:dunakeszi,/ar:90~120/netto-alapterulet:90~200/szobak:3~'''

'https://ingatlan.com/lista/eladó+ház+dunakeszi+90-120-mFt+90-200+3-szoba-felett\nhttps://dh.hu/elado-ingatlan/haz/Pest-megye/Dunakeszi/-/90-120-mFt/90-200-m2/3-szoba-tol\nhttps://ingatlan.jofogas.hu/pest/dunakeszi?max_price=120000000&max_size=200&min_price=90000000&min_size=90&ros=3&st=s\nhttps://www.startlak.hu/dunakeszi+elado+haz-hazresz+ar-90-120+szoba-3-+alapterulet-90-200\nhttps://www.ingatlannet.hu/eladó/ház/Dunakeszi?minPrice=90&maxPrice=120&roomMin=3&plotSizeMin=90&plotSizeMax=200\nhttps://www.oc.hu/ingatlanok/lista/oldalszam:12/rendezes:relevance/felhasznalas:elado/tipus:house/hely-ertek:dunakeszi/hely-id:dunakeszi,/ar:90~120/netto-alapterulet:90~200/szobak:3~'

In [224]:
ingatlan_df['detailed_page_url']

0     https://ingatlan.com/dunakeszi/elado+haz/csala...
1     https://ingatlan.com/dunakeszi/elado+haz/ikerh...
2     https://ingatlan.com/dunakeszi/elado+haz/csala...
3     https://ingatlan.com/dunakeszi/elado+lakas/teg...
4     https://ingatlan.com/dunakeszi-alag/elado+haz/...
                            ...                        
77    https://www.oc.hu/ingatlanok/DHZ068165?c=963a6bdf
78      https://www.oc.hu/ingatlanok/H404836?c=963a6bdf
79    https://www.oc.hu/ingatlanok/DHZ520994?c=963a6bdf
80      https://www.oc.hu/ingatlanok/H430043?c=963a6bdf
81    https://www.oc.hu/ingatlanok/DHZ049373?c=963a6bdf
Name: detailed_page_url, Length: 82, dtype: object

In [251]:
#HEATING SYSTEM
is_hs_gas=[]
for i in range(len(ingatlan_df)):
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='ingatlan.com':
        ingatlan_is_hs_gas=soup_list[i].find_all('dl', {'class': 'parameters'})
        print(ingatlan_df['detailed_page_url'].iloc[i])
        #print(str(ingatlan_is_hs_gas))
        #is_hs_gas.append(striphtml(str(ingatlan_is_hs_gas[5])))
        if "Fűtés" in str(ingatlan_is_hs_gas):
            #print("yes")
            print(striphtml(str(ingatlan_is_hs_gas).split("Fűtés")[1].split("</dd>")[0]))
            is_hs_gas.append(striphtml(str(ingatlan_is_hs_gas).split("Fűtés")[1].split("</dd>")[0]).replace("\n","").lower())
        else: 
            #print("no")
            print(np.nan)
            is_hs_gas.append(np.nan)
        
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='dh.hu':
        dh_is_hs_gas=soup_list[i].find_all('dd', {'class': 'col-xs-6'})
        #print(ingatlan_df['detailed_page_url'].iloc[i])
        #print(dh_is_hs_gas)
        #is_hs_gas.append(striphtml(str(dh_is_hs_gas[5])))
        
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='ingatlan.jofogas.hu':
        if soup_list[i].find_all('div', {'class': 'reParamBlock rePCAP rePCAP-heating_type'})!=[]:
            jofogas_is_hs_gas=soup_list[i].find_all('div', {'class': 'reParamBlock rePCAP rePCAP-heating_type'})
            print(ingatlan_df['detailed_page_url'].iloc[i])
            print(striphtml(str(jofogas_is_hs_gas).replace("Fűtés típusa:","")).replace("[","").replace("]",""))
            is_hs_gas.append(striphtml(str(jofogas_is_hs_gas).replace("Fűtés típusa:","")).lower())        
        else:
            print(np.nan)
            is_hs_gas.append(np.nan)
        
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='www.startlak.hu':
        startlak_is_hs_gas=soup_list[i].find_all('div', {'class': 'col-xs-12 col-sm-6 col-md-6 col-lg-6 amItem'})
        print(ingatlan_df['detailed_page_url'].iloc[i])
        #print(startlak_is_hs_gas)
        if "Fűtés" in striphtml(str(startlak_is_hs_gas)):
            print(striphtml(str(startlak_is_hs_gas)).split("Fűtés")[1].split(",")[0])
            is_hs_gas.append(striphtml(str(startlak_is_hs_gas)).split("Fűtés")[1].split(",")[0].lower())
        else: 
            print(np.nan)
            is_hs_gas.append(np.nan)
            
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='www.ingatlannet.hu':
        ingatlannet_is_hs_gas=soup_list[i].find_all('div', {'class': 'col-12 col-md-6 pt-1 pb-1'})
        print(ingatlan_df['detailed_page_url'].iloc[i])
        #print(striphtml(str(ingatlannet_is_hs_gas[4])).replace(" Fűtés",""))
        #print(ingatlannet_is_hs_gas)
        if "Fűtés" in str(ingatlannet_is_hs_gas):
            #print("yes")
            #print(str(ingatlannet_is_hs_gas))
            print(striphtml(str(ingatlannet_is_hs_gas).split("Fűtés")[1].split("</dd>")[0]))
            is_hs_gas.append(striphtml(str(ingatlannet_is_hs_gas).split("Fűtés")[1].split("</dd>")[0]))
        else: 
            print(np.nan)
            is_hs_gas.append(np.nan)
            
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='www.oc.hu':
        print(soup_list[i].find_all('ul', {'class': 'row'}))
        oc_is_hs_gas=soup_list[i].find_all('ul', {'class': 'row'})
        print(striphtml(str(oc_is_hs_gas)))
        print(ingatlan_df['detailed_page_url'].iloc[i])
        if "Fűtés:" in str(oc_is_hs_gas):
            print(str(oc_is_hs_gas).split("Fűtés:")[1].split("</span>")[1].replace("<span>",""))
            is_hs_gas.append(str(oc_is_hs_gas).split("Fűtés:")[1].split("</span>")[1].replace("<span>","").lower())
        else: 
            print(np.nan)
            is_hs_gas.append(np.nan)
        
        #print(striphtml(str(oc_is_hs_gas[3])).replace("Fűtés:",""))
        

https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/31768378

   
    gáz (cirko)
  
https://ingatlan.com/dunakeszi/elado+haz/ikerhaz/32352879

   
    gáz (cirko), padlófűtés
  
https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/31448814

   
    gáz (cirko), megújuló energia
  
https://ingatlan.com/dunakeszi/elado+lakas/tegla-epitesu-lakas/32238592

   
    padlófűtés
  
https://ingatlan.com/dunakeszi-alag/elado+haz/csaladi-haz/32188860

   
    gáz (cirko), egyéb
  
https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/31953740

   
    vegyes tüzelésű kazán
  
https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/32335824

   
    gáz (cirko)
  
https://ingatlan.com/dunakeszi-alag/elado+haz/csaladi-haz/32137729

   
    gáz (cirko), padlófűtés
  
https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/32240337

   
    gáz (cirko)
  
https://ingatlan.com/dunakeszi-alag/elado+haz/csaladi-haz/32392448

   
    hőszivattyú, padlófűtés
  
https://ingatlan.com/duna

<li class="col-md-12 td-separator"></li><li class="col-sm-6"><div><span class="room">szoba</span><span>15,76 m<sup>2</sup></span></div></li><li class="col-sm-6"><div><span class="room">szoba</span><span>11,73 m<sup>2</sup></span></div></li><li class="col-sm-6"><div><span class="room">szoba</span><span>12,67 m<sup>2</sup></span></div></li><li class="col-sm-6"><div><span class="room">szoba</span><span>24,46 m<sup>2</sup></span></div></li><li class="col-sm-6"><div><span class="room">fürdőszoba-wc</span><span>10,80 m<sup>2</sup></span></div></li><li class="col-sm-6"><div><span class="room">fürdőszoba-wc</span><span>3 m<sup>2</sup></span></div></li><li class="col-sm-6"><div><span class="room">garázs</span><span>25 m<sup>2</sup></span></div></li><li class="col-sm-6"><div><span class="room">háztartási helyiség</span><span>5 m<sup>2</sup></span></div></li><li class="col-sm-6"><div><span class="room">kamra</span><span>3 m<sup>2</sup></span></div></li><li class="col-sm-6"><div><span class="room"

[<ul class="row">
<li class="col-md-12 li-head">
                                Ingatlan adatok                            </li>
<li class="col-sm-6"><div><span>Méret (bruttó):</span><span>162 m<sup>2</sup></span></div></li><li class="col-sm-6"><div><span>Telekméret:</span><span>284 m<sup>2</sup></span></div></li><li class="col-sm-6"><div><span>Állapot:</span><span>Kiváló</span></div></li><li class="col-sm-6"><div><span>Fűtés:</span><span>gáz (cirko)</span></div></li><li class="col-sm-6"><div><span>Fürdőszobák száma:</span><span>2</span></div></li><li class="col-sm-6"><div><span>Házon belüli szintszám:</span><span>2</span></div></li><li class="col-sm-6"><div><span>Pince:</span><span>nincs</span></div></li><li class="col-sm-6"><div><span>Tájolás:</span><span>DNY</span></div></li> </ul>, <ul class="row">
<li class="col-md-12 li-head">
                                Helyiségek                            </li>
<li class="col-md-12 td-separator"></li><li class="col-sm-6"><div><span class=

gáz (cirko)


In [252]:
len(is_hs_gas)

82

In [253]:
ingatlan_df['is_hs_gas']=is_hs_gas
ingatlan_df

,source_page,detailed_page_url,price,building_year,is_train,is_bus,is_any_pt,garage,is_hs_gas,garden_size,is_nursery_near,is_school_near,street
0,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,116 M Ft,1981 és 2000 között,NaN,yes,NaN,nincs megadva,gáz (cirko),NaN,yes,yes,NaN
1,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/ikerh...,99 M Ft,2018,NaN,yes,NaN,udvari beálló - benne van az árban,"gáz (cirko), padlófűtés",NaN,yes,yes,lázár vilmos utca
2,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,91.2 M Ft,2021,NaN,yes,NaN,nincs megadva,"gáz (cirko), megújuló energia",NaN,yes,yes,pozsonyi utca
3,ingatlan.com,https://ingatlan.com/dunakeszi/elado+lakas/teg...,99 M Ft,2014,NaN,yes,NaN,"utca, közterület - ingyenes",padlófűtés,NaN,yes,yes,NaN
4,ingatlan.com,https://ingatlan.com/dunakeszi-alag/elado+haz/...,119 M Ft,1981 és 2000 között,NaN,yes,NaN,udvari beálló - benne van az árban,"gáz (cirko), egyéb",NaN,yes,yes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ068165?c=963a6bdf,"106,9 M Ft",NaN,NaN,NaN,NaN,NaN,gáz (cirko),NaN,NaN,NaN,NaN
78,www.oc.hu,https://www.oc.hu/ingatlanok/H404836?c=963a6bdf,"113,9 M Ft",NaN,NaN,NaN,NaN,NaN,vegyes (cirkó),NaN,NaN,NaN,NaN
79,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ520994?c=963a6bdf,99 M Ft,NaN,NaN,NaN,NaN,NaN,gáz (cirko),NaN,NaN,NaN,bazsanth vince utca
80,www.oc.hu,https://www.oc.hu/ingatlanok/H430043?c=963a6bdf,95 M Ft,NaN,NaN,yes,yes,NaN,vegyes (cirkó),NaN,yes,yes,NaN


In [278]:
#GARAGE
garage=[]
for i in range(len(ingatlan_df)):
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='ingatlan.com':
        ingatlan_garage=soup_list[i].find_all('dl', {'class': 'parameters'})
        print(ingatlan_df['detailed_page_url'].iloc[i])
        #print(str(ingatlan_is_hs_gas))
        #is_hs_gas.append(striphtml(str(ingatlan_is_hs_gas[5])))
        if "Parkolás" in str(ingatlan_garage):
            #print("yes")
            print(striphtml(str(ingatlan_garage).split("Parkolás")[1].split("</dd>")[0]))
            garage.append(striphtml(str(ingatlan_garage).split("Parkolás")[1].split("</dd>")[0]).replace("\n","").lower())
        else: 
            #print("no")
            print(np.nan)
            garage.append(np.nan)
        
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='dh.hu':
        dh_garage=soup_list[i].find_all('dd', {'class': 'col-xs-6'})
        #print(ingatlan_df['detailed_page_url'].iloc[i])
        #print(garage)
        #garage.append(striphtml(str(garage)))
        
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='ingatlan.jofogas.hu':
        if soup_list[i].find_all('div', {'class': 'reParamBlock rePCAP rePCAP-parking_type'})!=[]:
            jofogas_garage=soup_list[i].find_all('div', {'class': 'reParamBlock rePCAP rePCAP-parking_type'})
            print(ingatlan_df['detailed_page_url'].iloc[i])
            print(striphtml(str(jofogas_garage).replace("Parkolás:","")).replace("[","").replace("]","").lower())
            garage.append(striphtml(str(jofogas_garage).replace("Parkolás:","")).replace("[","").replace("]","").lower())    
        else:
            print(np.nan)
            garage.append(np.nan)
        
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='www.startlak.hu':
        startlak_garage=soup_list[i].find_all('div', {'class': 'amenities'})
        print(ingatlan_df['detailed_page_url'].iloc[i])
        #print(startlak_is_hs_gas)
        if "garázs" in striphtml(str(startlak_garage)):
            print("garázs")
            garage.append("garázs")
        elif "kocsibeálló" in striphtml(str(startlak_garage)):
            print("kocsibeálló")
            garage.append("kocsibeálló")
        else: 
            print(np.nan)
            garage.append(np.nan)
            
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='www.ingatlannet.hu':
        ingatlannet_garage=soup_list[i].find_all('div', {'class': 'col-12 col-md-6 pt-1 pb-1'})
        print(ingatlan_df['detailed_page_url'].iloc[i])
        #print(ingatlannet_garage)
        if "Parkolás" in str(ingatlannet_garage):
            #print("yes")
            #print(str(ingatlannet_garage))
            print(striphtml((str(ingatlannet_garage).split("Parkolás")[1].split("</dd>")[0])))
            garage.append(striphtml((str(ingatlannet_garage).split("Parkolás")[1].split("</dd>")[0])))
        else: 
            print(np.nan)
            garage.append(np.nan)
            
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='www.oc.hu':
        oc_garage=soup_list[i].find_all('ul', {'class': 'row'})
        #print(striphtml(str(oc_garage)))
        print(ingatlan_df['detailed_page_url'].iloc[i])
        if "garázs:" in str(oc_garage):
            print("yes")
            #print(str(oc_garage).split("garázs:")[1].split("</span>")[1].replace("<span>",""))
            garage.append(str(oc_garage).split("garázs:")[1].split("</span>")[1].replace("<span>","").lower())
        else: 
            print(np.nan)
            garage.append(np.nan)
        

https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/31768378

   
    nincs megadva
  
https://ingatlan.com/dunakeszi/elado+haz/ikerhaz/32352879

   
    udvari beálló - benne van az árban
  
https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/31448814

   
    nincs megadva
  
https://ingatlan.com/dunakeszi/elado+lakas/tegla-epitesu-lakas/32238592

   
    utca, közterület - ingyenes
  
https://ingatlan.com/dunakeszi-alag/elado+haz/csaladi-haz/32188860

   
    udvari beálló - benne van az árban
  
https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/31953740

   
    udvari beálló - benne van az árban
  
https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/32335824

   
    nincs megadva
  
https://ingatlan.com/dunakeszi-alag/elado+haz/csaladi-haz/32137729

   
    teremgarázs hely - benne van az árban
  
https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/32240337

   
    udvari beálló - benne van az árban
  
https://ingatlan.com/dunakeszi-alag/elado+haz/

In [279]:
len(garage)

82

In [280]:
'''garage=[]
for i in range(len(soup_list)):
    #print(soup_list[i])
    mystring = striphtml(str(soup_list[i]).lower())
    #print(mystring)
    garage_op=re.compile(r'(garázs.*)(?![^<>]*>)') 
    garage_op=garage_op.findall(mystring)
    #print(garage_op)
    if garage_op!=[] and "garagesforrent" not in garage_op and "kiadó garázs" not in garage_op:
        garage.append("yes")
    else:
        garage.append("no")
    
len(garage)'''

'garage=[]\nfor i in range(len(soup_list)):\n    #print(soup_list[i])\n    mystring = striphtml(str(soup_list[i]).lower())\n    #print(mystring)\n    garage_op=re.compile(r\'(garázs.*)(?![^<>]*>)\') \n    garage_op=garage_op.findall(mystring)\n    #print(garage_op)\n    if garage_op!=[] and "garagesforrent" not in garage_op and "kiadó garázs" not in garage_op:\n        garage.append("yes")\n    else:\n        garage.append("no")\n    \nlen(garage)'

In [281]:
ingatlan_df['garage']=garage
ingatlan_df
#ingatlan_df[ingatlan_df['is_train']=='yes']

,source_page,detailed_page_url,price,building_year,is_train,is_bus,is_any_pt,garage,is_hs_gas,garden_size,is_nursery_near,is_school_near,street
0,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,116 M Ft,1981 és 2000 között,NaN,yes,NaN,nincs megadva,gáz (cirko),1135 m²,yes,yes,NaN
1,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/ikerh...,99 M Ft,2018,NaN,yes,NaN,udvari beálló - benne van az árban,"gáz (cirko), padlófűtés",500 m²,yes,yes,lázár vilmos utca
2,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,91.2 M Ft,2021,NaN,yes,NaN,nincs megadva,"gáz (cirko), megújuló energia",404 m²,yes,yes,pozsonyi utca
3,ingatlan.com,https://ingatlan.com/dunakeszi/elado+lakas/teg...,99 M Ft,2014,NaN,yes,NaN,"utca, közterület - ingyenes",padlófűtés,NaN,yes,yes,NaN
4,ingatlan.com,https://ingatlan.com/dunakeszi-alag/elado+haz/...,119 M Ft,1981 és 2000 között,NaN,yes,NaN,udvari beálló - benne van az árban,"gáz (cirko), egyéb",562 m²,yes,yes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ068165?c=963a6bdf,"106,9 M Ft",NaN,NaN,NaN,NaN,NaN,gáz (cirko),284 m²,NaN,NaN,NaN
78,www.oc.hu,https://www.oc.hu/ingatlanok/H404836?c=963a6bdf,"113,9 M Ft",NaN,NaN,NaN,NaN,NaN,vegyes (cirkó),1 591 m²,NaN,NaN,NaN
79,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ520994?c=963a6bdf,99 M Ft,NaN,NaN,NaN,NaN,NaN,gáz (cirko),1 135 m²,NaN,NaN,bazsanth vince utca
80,www.oc.hu,https://www.oc.hu/ingatlanok/H430043?c=963a6bdf,95 M Ft,NaN,NaN,yes,yes,NaN,vegyes (cirkó),993 m²,yes,yes,NaN


In [282]:
#BUILDING_YEAR
building_year=[]
for i in range(len(ingatlan_df)):
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='ingatlan.com':
        ingatlan_building_year=soup_list[i].find_all('dl', {'class': 'parameters'})
        print(ingatlan_df['detailed_page_url'].iloc[i])
        #print(str(ingatlan_is_hs_gas))
        if "Építés éve" in str(ingatlan_building_year):
            #print("yes")
            print(striphtml(str(ingatlan_building_year).split("Építés éve")[1].split("</dd>")[0]))
            building_year.append(striphtml(str(ingatlan_building_year).split("Építés éve")[1].split("</dd>")[0]).replace("\n","").lower())
        else: 
            #print("no")
            print(np.nan)
            building_year.append(np.nan)
        
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='dh.hu':
        dh_building_year=soup_list[i].find_all('dd', {'class': 'col-xs-6'})
        #print(ingatlan_df['detailed_page_url'].iloc[i])
        #print(building_year)
        #building_year.append(striphtml(str(building_year)))
        
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='ingatlan.jofogas.hu':
        building_year.append(np.nan)
        
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='www.startlak.hu':
        building_year.append(np.nan)
            
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='www.ingatlannet.hu':
        ingatlannet_building_year=soup_list[i].find_all('div', {'class': 'col-12 col-md-6 pt-1 pb-1'})
        print(ingatlan_df['detailed_page_url'].iloc[i])
        #print(ingatlannet_building_year)
        if "Építés éve" in str(ingatlannet_building_year):
            #print("yes")
            #print(str(ingatlannet_building_year))
            print(striphtml((str(ingatlannet_building_year).split("Építés éve")[1].split("</dd>")[0])))
            building_year.append(striphtml((str(ingatlannet_building_year).split("Építés éve")[1].split("</dd>")[0])))
        else: 
            print(np.nan)
            building_year.append(np.nan)
            
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='www.oc.hu':
        oc_building_year=soup_list[i].find_all('ul', {'class': 'row'})
        #print(striphtml(str(oc_building_year)))
        print(ingatlan_df['detailed_page_url'].iloc[i])
        if "Építés éve:" in str(oc_building_year):
            print("yes")
            #print(str(oc_building_year).split("garázs:")[1].split("</span>")[1].replace("<span>",""))
            building_year.append(str(oc_building_year).split("Építés éve:")[1].split("</span>")[1].replace("<span>","").lower())
        else: 
            print(np.nan)
            building_year.append(np.nan)
        

https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/31768378

   
    1981 és 2000 között
  
https://ingatlan.com/dunakeszi/elado+haz/ikerhaz/32352879

   
    2018
  
https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/31448814

   
    2021
  
https://ingatlan.com/dunakeszi/elado+lakas/tegla-epitesu-lakas/32238592

   
    2014
  
https://ingatlan.com/dunakeszi-alag/elado+haz/csaladi-haz/32188860

   
    1981 és 2000 között
  
https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/31953740

   
    1950 és 1980 között
  
https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/32335824

   
    2001 és 2010 között
  
https://ingatlan.com/dunakeszi-alag/elado+haz/csaladi-haz/32137729

   
    1950 és 1980 között
  
https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/32240337

   
    2001 és 2010 között
  
https://ingatlan.com/dunakeszi-alag/elado+haz/csaladi-haz/32392448
nan
https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/30444220

   
    1981 és 2000 közö

In [283]:
len(building_year)

82

In [284]:
ingatlan_df['building_year']=building_year
ingatlan_df

,source_page,detailed_page_url,price,building_year,is_train,is_bus,is_any_pt,garage,is_hs_gas,garden_size,is_nursery_near,is_school_near,street
0,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,116 M Ft,1981 és 2000 között,NaN,yes,NaN,nincs megadva,gáz (cirko),1135 m²,yes,yes,NaN
1,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/ikerh...,99 M Ft,2018,NaN,yes,NaN,udvari beálló - benne van az árban,"gáz (cirko), padlófűtés",500 m²,yes,yes,lázár vilmos utca
2,ingatlan.com,https://ingatlan.com/dunakeszi/elado+haz/csala...,91.2 M Ft,2021,NaN,yes,NaN,nincs megadva,"gáz (cirko), megújuló energia",404 m²,yes,yes,pozsonyi utca
3,ingatlan.com,https://ingatlan.com/dunakeszi/elado+lakas/teg...,99 M Ft,2014,NaN,yes,NaN,"utca, közterület - ingyenes",padlófűtés,NaN,yes,yes,NaN
4,ingatlan.com,https://ingatlan.com/dunakeszi-alag/elado+haz/...,119 M Ft,1981 és 2000 között,NaN,yes,NaN,udvari beálló - benne van az árban,"gáz (cirko), egyéb",562 m²,yes,yes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ068165?c=963a6bdf,"106,9 M Ft",NaN,NaN,NaN,NaN,NaN,gáz (cirko),284 m²,NaN,NaN,NaN
78,www.oc.hu,https://www.oc.hu/ingatlanok/H404836?c=963a6bdf,"113,9 M Ft",NaN,NaN,NaN,NaN,NaN,vegyes (cirkó),1 591 m²,NaN,NaN,NaN
79,www.oc.hu,https://www.oc.hu/ingatlanok/DHZ520994?c=963a6bdf,99 M Ft,NaN,NaN,NaN,NaN,NaN,gáz (cirko),1 135 m²,NaN,NaN,bazsanth vince utca
80,www.oc.hu,https://www.oc.hu/ingatlanok/H430043?c=963a6bdf,95 M Ft,NaN,NaN,yes,yes,NaN,vegyes (cirkó),993 m²,yes,yes,NaN


In [285]:
len(ingatlan_df[ingatlan_df['source_page']=='ingatlan.com'])

20

In [286]:
len(ingatlan_df[ingatlan_df['source_page']=='ingatlan.jofogas.hu'])

23

In [287]:
len(ingatlan_df[ingatlan_df['source_page']=='www.ingatlannet.hu'])

11

In [288]:
len(ingatlan_df[ingatlan_df['source_page']=='www.startlak.hu'])

20

In [289]:
len(ingatlan_df[ingatlan_df['source_page']=='www.oc.hu'])

8

In [290]:
#GARDEN_SIZE
garden_size=[]
for i in range(len(ingatlan_df)):
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='ingatlan.com':
        if soup_list[i].find_all('div', {'class': 'parameter'})!=[]:
            ingatlan_garden_size=soup_list[i].find_all('div', {'class': 'parameter'})
            print(ingatlan_df['detailed_page_url'].iloc[i])
            if "Telekterület" in str(ingatlan_garden_size[2]):
                print(striphtml(str(ingatlan_garden_size[2])).replace("Telekterület",""))
                garden_size.append(striphtml(str(ingatlan_garden_size[2])).replace("Telekterület","").replace("\n",""))
            else:
                garden_size.append(np.nan)
        else: 
            #print("no")
            #print(np.nan)
            garden_size.append(np.nan)
        
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='dh.hu':
        dh_garden_size=soup_list[i].find_all('dd', {'class': 'col-xs-6'})
        #print(ingatlan_df['detailed_page_url'].iloc[i])
        #print(dh_garden_size)
        #garden_size.append(striphtml(str(dh_garden_size[5])))
        
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='ingatlan.jofogas.hu':
        print(ingatlan_df['detailed_page_url'].iloc[i])
        print(np.nan)
        garden_size.append(np.nan)
        
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='www.startlak.hu':
        startlak_garden_size=soup_list[i].find_all('div', {'class': 'col-xs-12 col-sm-6 col-md-6 col-lg-6 amItem'})
        print(ingatlan_df['detailed_page_url'].iloc[i])
        #print(startlak_garden_size)
        if "Telekterület" in striphtml(str(startlak_garden_size)):
            print(striphtml(str(startlak_garden_size)).split("Telekterület")[1].split(",")[0])
            garden_size.append(striphtml(str(startlak_garden_size)).split("Telekterület")[1].split(",")[0].lower())
        else: 
            print(np.nan)
            garden_size.append(np.nan)
            
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='www.ingatlannet.hu':
        if soup_list[i].find_all('ul', {'class': 'list-inline mb-1'})!=[]:
            ingatlannet_garden_size=soup_list[i].find_all('ul', {'class': 'list-inline mb-1'})
            print(ingatlan_df['detailed_page_url'].iloc[i])
            print(striphtml(str(ingatlannet_garden_size).split("szoba")[1]).split("m²")[0].strip()+" m²")
            garden_size.append(striphtml(str(ingatlannet_garden_size).split("szoba")[1]).split("m²")[0].strip()+" m²")
        else: 
            print(np.nan)
            garden_size.append(np.nan)
            
    if str(ingatlan_df['detailed_page_url'].iloc[i])[8:].split("/")[0]=='www.oc.hu':
        oc_garden_size=soup_list[i].find_all('ul', {'class': 'row'})
        #print(striphtml(str(oc_garden_size)))
        print(ingatlan_df['detailed_page_url'].iloc[i])
        if "Telekméret:" in str(oc_garden_size):
            print(str(oc_garden_size).split("Telekméret:")[1].split("</span>")[1].replace("<span>","").strip().replace("m<sup>2</sup>","m²"))
            garden_size.append(str(oc_garden_size).split("Telekméret:")[1].split("</span>")[1].replace("<span>","").strip().replace("m<sup>2</sup>","m²"))
        else: 
            print(np.nan)
            garden_size.append(np.nan)
        
        #print(striphtml(str(oc_garden_size[3])).replace("Fűtés:",""))
        

https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/31768378

        
       1135 m²
https://ingatlan.com/dunakeszi/elado+haz/ikerhaz/32352879

        
       500 m²
https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/31448814

        
       404 m²
https://ingatlan.com/dunakeszi/elado+lakas/tegla-epitesu-lakas/32238592
https://ingatlan.com/dunakeszi-alag/elado+haz/csaladi-haz/32188860

        
       562 m²
https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/31953740

        
       1571 m²
https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/32335824

        
       571 m²
https://ingatlan.com/dunakeszi-alag/elado+haz/csaladi-haz/32137729

        
       1387 m²
https://ingatlan.com/dunakeszi-revdulo/elado+haz/csaladi-haz/32240337

        
       571 m²
https://ingatlan.com/dunakeszi-alag/elado+haz/csaladi-haz/32392448

        
       515 m²
https://ingatlan.com/dunakeszi/elado+haz/csaladi-haz/30444220

        
       564 m²
https://ingatlan.com/dunakeszi

In [291]:
len(garden_size)

82

In [296]:
ingatlan_df['garden_size_m2']=garden_size
ingatlan_df

,source_page,detailed_page_url,price,building_year,is_train,is_bus,is_any_pt,garage,is_hs_gas,garden_size_m2,is_nursery_near,is_school_near,street
0,NaN,https://ingatlan.com/dunakeszi/elado+haz/csala...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1135 m²,NaN,NaN,NaN
1,NaN,https://ingatlan.com/dunakeszi/elado+haz/ikerh...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500 m²,NaN,NaN,NaN
2,NaN,https://ingatlan.com/dunakeszi/elado+haz/csala...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,404 m²,NaN,NaN,NaN
3,NaN,https://ingatlan.com/dunakeszi/elado+lakas/teg...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,https://ingatlan.com/dunakeszi-alag/elado+haz/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,562 m²,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,NaN,https://www.oc.hu/ingatlanok/DHZ068165?c=963a6bdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,284 m²,NaN,NaN,NaN
78,NaN,https://www.oc.hu/ingatlanok/H404836?c=963a6bdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 591 m²,NaN,NaN,NaN
79,NaN,https://www.oc.hu/ingatlanok/DHZ520994?c=963a6bdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 135 m²,NaN,NaN,NaN
80,NaN,https://www.oc.hu/ingatlanok/H430043?c=963a6bdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,993 m²,NaN,NaN,NaN


In [293]:
writer = pd.ExcelWriter('ingatlan_scraper_result_dunakeszi.xlsx')
#write dataframe to excel
ingatlan_df.to_excel(writer)
# save the excel
writer.save()

In [294]:
ingatlan_df[ingatlan_df['is_train']=='yes']

,source_page,detailed_page_url,price,building_year,is_train,is_bus,is_any_pt,garage,is_hs_gas,garden_size,is_nursery_near,is_school_near,street
6,ingatlan.com,https://ingatlan.com/dunakeszi-revdulo/elado+h...,117 M Ft,2001 és 2010 között,yes,yes,NaN,nincs megadva,gáz (cirko),571 m²,yes,yes,NaN
15,ingatlan.com,https://ingatlan.com/dunakeszi-gyartelep/elado...,119.9 M Ft,1981 és 2000 között,yes,yes,NaN,nincs megadva,gáz (cirko),550 m²,yes,yes,NaN
18,ingatlan.com,https://ingatlan.com/dunakeszi-revdulo/elado+h...,103 M Ft,2001 és 2010 között,yes,yes,NaN,nincs megadva,gáz (cirko),571 m²,yes,yes,NaN
65,www.ingatlannet.hu,https://www.ingatlannet.hu/ingatlan/Dunakeszi/...,98 millió Ft,\n...,yes,NaN,NaN,\n...,\n...,722 m²,yes,yes,jácint utca
69,www.ingatlannet.hu,https://www.ingatlannet.hu/ingatlan/Dunakeszi/...,99 millió Ft,\n...,yes,NaN,NaN,\n...,\n...,515 m²,NaN,NaN,jácint utca
71,www.ingatlannet.hu,https://www.ingatlannet.hu/ingatlan/Dunakeszi/...,115 millió Ft,\n...,yes,NaN,NaN,\n...,\n...,550 m²,NaN,NaN,NaN
74,www.oc.hu,https://www.oc.hu/ingatlanok/H434067?c=963a6bdf,119 M Ft,NaN,yes,NaN,yes,NaN,gáz (cirko),550 m²,NaN,NaN,NaN
75,www.oc.hu,https://www.oc.hu/ingatlanok/H430331?c=963a6bdf,"102,5 M Ft",NaN,yes,yes,yes,NaN,gáz (cirko),276 m²,yes,yes,NaN
